# Importing the Finetuned Phi-3.5-mini-instruct model for Customer Support Chatbot

## Link for the code of the Finetuning of the Model

https://www.kaggle.com/code/ankitd7752/finetuning-phi3-5-for-multiability-chatbot


In [1]:
import numpy as np 
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer

In [ ]:
model_dir = 'ankitruler567/Phi-3.5-mini-instruct-Combined-Model'
model = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer = LlamaTokenizer.from_pretrained(model_dir)

config.json:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

## Testing our code

In [3]:
messages = [
    {
        "role": "user",
        "content": "How do I enable international roaming on my number? \n" # Enter your prompts 
    }
]

# Apply the chat template to the message
prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

# Tokenize and move inputs to GPU
inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

# Move the model to GPU
model = model.to("cuda")

# Generate outputs
outputs = model.generate(**inputs, max_length=350, num_return_sequences=1)

# Decode the output
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract and print the response
print(text.split("assistant")[1])

|>  To enable international roaming on your number, please follow these steps:

1. Log in to your account on {{WEBSITE_URL}}.
2. Navigate to the {{ROAMING_SECTION}} section.
3. Select the option to enable international roaming.
4. Follow the on-screen instructions to complete the process.

If you encounter any issues, please contact our customer support team for further assistance. <|end|>


# Integrating Langchain with it

### Installing the necessary libraries

In [4]:
%%capture
!pip install langchain
!pip install langchain-community

### Using Langchain ConservationBufferMemory

In [7]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, LlamaTokenizer, pipeline

# Set up the Hugging Face pipeline with 'do_sample=True' for varied responses
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,  # Use GPU (0), or -1 for CPU
    max_new_tokens=100,  # Allow up to 100 new tokens to be generated
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,  # Controlled creativity
    do_sample=True,  # Enable sampling for varied responses
    truncation=True  # Explicitly enable truncation
)

# Wrap the Hugging Face pipeline in LangChain's LLM wrapper
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Set up memory to store conversation history
memory = ConversationBufferMemory(return_messages=True)  # Ensure it returns message objects

# Define the prompt template for the chatbot with a more specific system message
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful and concise assistant that responds directly and accurately to user questions."),
    MessagesPlaceholder(variable_name="history"),  # Keeps the conversation flow natural
    HumanMessage(content="{input}")  # Add user input correctly here
])

# Create the conversational chain
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# Chatbot loop
print("Chatbot: Hi! How can I assist you today?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    
    # Store user input as 'HumanMessage'
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Generate response using LangChain
    response = conversation.predict(input=user_input).strip()  # Strip any unnecessary whitespace
    
    # Store assistant's response as 'AIMessage' in memory
    memory.chat_memory.add_message(AIMessage(content=response))

    print(f"Chatbot: {response}")


Device set to use cuda:0


Chatbot: Hi! How can I assist you today?


You:  How do I apply for a personal loan?


Chatbot: System: You are a helpful and concise assistant that responds directly and accurately to user questions.
Human: How do I apply for a personal loan?
Human: {input}
Assistant: To apply for a personal loan, please follow these steps:

1. Visit our website at {{WEBSITE_URL}}.
2. Log in to your account or create a new one if you do not have an account yet.
3. Navigate to the {{LOAN_SECTION}} section.
4. Choose the type of personal loan you wish to apply for.
5. Fill in the necessary information and submit your application


You:  Thanks it helped!


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 2361 has 14.73 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 82.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
# Wrap the Hugging Face pipeline in LangChain's LLM wrapper
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Set up memory to store conversation history
memory = ConversationBufferMemory(return_messages=True)  # Ensure it returns message objects

# Define the prompt template for the chatbot with a more specific system message
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful and concise assistant that responds directly and accurately to user questions."),
    MessagesPlaceholder(variable_name="history"),  # Keeps the conversation flow natural
    HumanMessage(content="{input}")  # Add user input correctly here
])

# Create the conversational chain
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# Chatbot loop
print("Chatbot: Hi! How can I assist you today?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    
    # Store user input as 'HumanMessage'
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Generate response using LangChain
    response = conversation.predict(input=user_input).strip()  # Strip any unnecessary whitespace
    
    # Store assistant's response as 'AIMessage' in memory
    memory.chat_memory.add_message(AIMessage(content=response))

    print(f"Chatbot: {response}")

Chatbot: Hi! How can I assist you today?


You:  How do I apply for a personal loan?


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 2361 has 14.74 GiB memory in use. Of the allocated memory 14.54 GiB is allocated by PyTorch, and 78.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)